##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 訓練後の整数量子化

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/lite/performance/post_training_integer_quant"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/performance/post_training_integer_quant.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/performance/post_training_integer_quant.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## 概要

現在、[TensorFlow Lite](https://www.tensorflow.org/lite/) は TensorFlow から TensorFlow Lite のフラットバッファー形式へ変換する際に、
モデルのすべての値(重みと活性化関数)を8ビット整数へ変換することをサポートしています。
これによりモデルの大きさが4倍小さくなり、CPU上での性能が3~4倍改善します。
さらに、この完全に量子化されたモデルは整数演算のみに対応したハードウェアアクセラレータで使用できます。

重みのみを8bitで保持する [訓練後の "オン・ザ・フライ" 量子化](https://colab.sandbox.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/tutorials/post_training_quant.ipynb) とは対照的に、
この手法はモデル変換中に静的にすべての重み `と` 活性化関数を量子化します。

このチュートリアルでは、MNIST モデルをスクラッチから訓練し、その精度を TensorFlow で検査しそれから、モデルを完全に量子化された TensorFlow Lite のフラットバッファーに変換します。
最後に変換されたモデルの精度を検査し、それを元の浮動小数点数モデルと比較します。

## MNIST モデルをビルドする

### セットアップ

In [ ]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pathlib

### モデルを訓練してエクスポートする

In [ ]:
# MNIST データセットを読み込みます
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 入力画像を正規化することにより、各画素値は0から1の間の値になります
train_images = train_images / 255.0
test_images = test_images / 255.0

# モデル構造を定義します
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# 数値分類モデルを訓練します
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=1,
  validation_data=(test_images, test_labels)
)

モデルを1エポックだけ訓練するので、この訓練は時間がかかりませんし、およそ96%の精度に達します。

### TensorFlow Lite モデルに変換する

Python の [TFLiteConverter](https://www.tensorflow.org/lite/convert/python_api) を使って、訓練されたモデルを TensorFlow Lite のモデルに変換することができます。

`TFLiteConverter` を使ってモデルを読み込みましょう:

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

`.tflite` file に書き出します:

In [ ]:
tflite_models_dir = pathlib.Path("/tmp/mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
tflite_model_file = tflite_models_dir/"mnist_model.tflite"
tflite_model_file.write_bytes(tflite_model)

さて、`.tfile` に変換された訓練された MNIST モデルを取得しましたが、それはまだすべてのパラメータに対して単精度浮動小数点数を使っています。

今度は量子化を使って、モデルを再度変換しましょう

#### 量子化を使って変換する

まず、最初に大きさを最適化するために `optimizations` フラグをセットします。

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

さて、活性化関数の正確なダイナミックレンジをもつ量子化値を作るために、
代表的なデータセットを提供する必要があります。

In [ ]:
mnist_train, _ = tf.keras.datasets.mnist.load_data()
images = tf.cast(mnist_train[0], tf.float32) / 255.0
mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    yield [input_value]

converter.representative_dataset = representative_data_gen

最後に、モデルを TensorFlow Lite 形式に変換します:

In [ ]:
tflite_model_quant = converter.convert()
tflite_model_quant_file = tflite_models_dir/"mnist_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

結果として得られるファイルは、およそ `1/4' のサイズになっていることに注意してください。

In [ ]:
!ls -lh {tflite_models_dir}

モデルは完全に量子化されているでしょう。
しかし、もしモデルが TensorFlow Lite が量子化できない演算を含む場合には、
それらの演算は浮動小数点数のまま残されています。
これにより小さく効率的なモデルを作り上げることができますが、
そのモデルは完全な整数量子化を必要とするいくつかの機械学習アクセラレータとは互換性がないでしょう。
また、デフォルトでは、変換されたモデルはまだ浮動小数点数の入力と出力を使用しており、
いくつかのアクセラレータとは互換性がありません。

変換されたモデルが完全に量子化されているか確認するため(量子化されていない演算に遭遇したら変換器にエラーを投げさせる)、
整数をモデルの入力と出力に使用して、モデルに追加設定を行ってからもう一度変換するする必要があります:


In [ ]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()
tflite_model_quant_file = tflite_models_dir/"mnist_model_quant_io.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

このサンプルでは、結果として得られるモデルの大きさはおなじままです。
すべての演算が量子化されるからです。
しかし、この新しいモデルは量子化された入力と出力を使用し、Coral Edge TPU のような多くのアクセラレータと互換があります。

以降の節では、2つのTensorFlow Liteモデルを扱うことに注意してください:
`tflite_model_file` は、浮動小数点数パラメータをまだ使用している変換モデルで、
`tflite_model_quant_file` は、uint8の入力と出力を含む完全に整数量子化されて変換されたおなじモデルです。

## TFLite モデルを実行する

TensorFlow Lite モデルを Python の TensorFlow Lite
インタープリタを使って実行します。

### モデルをインタープリタに読み込む

In [ ]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

In [ ]:
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()
input_index_quant = interpreter_quant.get_input_details()[0]["index"]
output_index_quant = interpreter_quant.get_output_details()[0]["index"]

### 1つの画像に対してモデルを検証する

まず浮動小数点数モデルで検証します:

In [ ]:
test_image = np.expand_dims(test_images[0], axis=0).astype(np.float32)

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
interpreter.set_tensor(input_index, test_image)
interpreter.invoke()
predictions = interpreter.get_tensor(output_index)

In [ ]:
import matplotlib.pylab as plt

plt.imshow(test_images[0])
template = "True:{true}, predicted:{predict}"
_ = plt.title(template.format(true= str(test_labels[0]),
                              predict=str(np.argmax(predictions[0]))))
plt.grid(False)

今度は量子化されたモデル(uint8データを使用する)を検証します:

In [ ]:
input_index = interpreter_quant.get_input_details()[0]["index"]
output_index = interpreter_quant.get_output_details()[0]["index"]
interpreter_quant.set_tensor(input_index, test_image)
interpreter_quant.invoke()
predictions = interpreter_quant.get_tensor(output_index)

In [ ]:
plt.imshow(test_images[0])
template = "True:{true}, predicted:{predict}"
_ = plt.title(template.format(true= str(test_labels[0]),
                              predict=str(np.argmax(predictions[0]))))
plt.grid(False)

### モデルを評価する

In [ ]:
# "test" データセットを使って TF Lite モデルを評価するヘルパー関数
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # "test" データセットの画像ごとに予測を実行する
  prediction_digits = []
  for test_image in test_images:
    # 前処理: バッチの次元を追加し、単精度浮動小数点数に変換し、
    # モデルの入力データフォーマットに合わせます
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # 推論を実行します。
    interpreter.invoke()

    # 後処理: バッチの次元を取り除き、最も出力の高い数字を見つける
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  # 予測結果を正解ラベルと比較し、精度を計算します。
  accurate_count = 0
  for index in range(len(prediction_digits)):
    if prediction_digits[index] == test_labels[index]:
      accurate_count += 1
  accuracy = accurate_count * 1.0 / len(prediction_digits)

  return accuracy

In [ ]:
print(evaluate_model(interpreter))

uint8データを使用した完全に量子化されたモデルで評価を繰り返します:

In [ ]:
# NOTE: Colab はサーバーのCPUで実行します。そして、TensorFlow Lite は現在、
# 非常に最適化されたサーバーCPUカーネルはありません。そのためこの箇所は
# 上記の浮動小数点数インタープリタよりも遅いかもしれません。しかし、モバイルCPUでは、
# かなりの高速化を観測できるでしょう。

print(evaluate_model(interpreter_quant))

このサンプルでは、上記の浮動小数点数モデルと比較して、精度にほとんど差がないモデルを完全に量子化できました。